# Structures for the implementation of Stream Ciphers

## Introduction

**Stream Ciphers** are a category of symmetric key cryptographic algorithms that encrypt data by operating on it bitwise (or bytewise), combining each piece of plaintext with a pseudorandom sequence known as **keystream**. Encryption involves applying an XOR operation between each plaintext bit and the corresponding keystream bit.
In this document are shown the three data stuctures related to the design and analysis of a stream cipher:

- The **Linear Feedback Shift Register class(LFSR)**, an hardware-based mechanism for producing pseudorandom bit sequences;

- The **Berlekamp-Massey algorithm**, used to find the minimal-length LFSR capable of generating a given bit sequence;

- The **Alternating-Step Generator**, which employs three interconnected LFSRs to introduce nonlinearity and irregular timing into the keystream.

It's also important to mention the **Bits class**, which is a mutable data structure for managing sequences of bits.
Lastly a final task has been dedicated to evaluating the cipher's security against a known-plaintext attack (KPA) scenario.

In [1]:
from bits import Bits
from lfsr import LFSR, berlekamp_massey
from bitgenerator import AlternatingStep

## LFSR

The **Linear Feedback Shift Register** (LFSR) is a type of shift register used to generate pseudorandom binary sequences in a **cyclic** and deterministic manner. At each clock cycle, the bits in the register are shifted, and a new feedback bit is inserted. This feedback bit is computed as a linear function (XOR) of selected bits from the current register state. The bit positions involved in this computation are known as "taps", determined by a feedback polynomial over the Galois Field 2.
The operation of an LFSR is defined by its initial state, called "seed", from which each subsequent state is uniquely and predictably determined. Due to the finite number of possible states, the sequence produced by an LFSR eventually becomes periodic, repeating after a certain number of steps. In cryptographic applications, LFSRs are often employed for key expansion, where the seed serves as the initial key material and the generated sequence provides a longer keystream for encryption purposes. The combination of simplicity, cyclic behavior, and predictable pseudorandomness makes LFSRs fundamental components in various digital systems.

In [3]:
# TEST LFSR
lfsr = LFSR(poly={5,2,1,0}, state=0b00101)
print(lfsr)

# Run steps to get a defined amount of output bits
lfsr = LFSR(poly={5,2,1,0}, state=0b00101)
out_bits = Bits(lfsr.output) + lfsr.run_steps(7)
print("\n8 steps result:", out_bits)

# Full cycle length
lfsr = LFSR(poly={5,2,1,0}, state=0b00101)
cycle_bits = lfsr.cycle()
print("Full cycle length:", len(cycle_bits))

LFSR(poly={5,2,1,0}, length=5, state=00101)

8 steps result: 10100111
Full cycle length: 7


## Berlekamp-Massey Algorithm

The **Berlekamp-Massey** algorithm plays a crucial role in cryptoanalysis by determining the shortest linear feedback polynomial that can reproduce a given pseudorandom binary sequence, deducing the structure of the LFSR and recovering its internal state or key.
It works by examining the sequence bit by bit sequentially. At each step, it checks if the current feedback polynomial correctly predicts the next bit. When a discrepancy occurs between the expected and actual bit, the algorithm updates the feedback polynomial. This adjustment is achieved by combining the current polynomial with a previously stored version that is shifted by an appropriate number of positions. This allows the algorithm to maintain past correct predictions while correcting the new error.

When a sequence is not fully generated — meaning the LFSR does not utilize all possible states — the algorithm can efficiently derive the minimal polynomial for the shortest LFSR capable of producing the sequence. In doing so, it not only supports the efficient generation of long pseudorandom sequences with minimal resources but also exposes vulnerabilities that can be exploited to compromise LFSR-based cryptographic systems.

In [16]:
# TEST BERLEKAMP-MASSEY
with open(file='binary_sequence.bin', mode='rb') as f:
   txt = f.read()

bits = Bits(txt)

recovered_poly = berlekamp_massey(bits)
print(f"Recovered feedback polynomial of order {recovered_poly[1]}:", list(i for i, item in enumerate(recovered_poly[0]) if item))

Recovered feedback polynomial of order 18: [0, 7, 18]


## Alternating-Step Generator

The **Alternating-Step Generator** is a structure used to produce a pseudorandom output sequence by using several LFSRs. The reason behind it's implementation is that it introduces non-linearity, thus increasing unpredictability. In particular, there are two **data LFSRs** for the generation of output bits (**LFSR$_0$ and LFSR$_1$**) and a **control LFSR$_C$** that determines which one between them needs to be clocked. The clocked register will provide as output the next bit of the keystream.

In [20]:
# TEST ALTERNATING STEP GENERATOR

# LFSRc = {5,2,0}
# LFSR0 = {3,1,0}  LFSR1 = {4,1,0}
gen = AlternatingStep()  # keeping default values for the three LFSRs and states

output_sequence = []
for _ in range(12):
    output_sequence.append(next(gen))

print("Output sequnce of the generator, first 12 values:", Bits(output_sequence))

Output sequnce of the generator, first 12 values: 000111010101


## Known Plaintext Attack (KPA)

A **Known-Plaintext Attack** (KPA) occurs when an attacker has access to both a segment of plaintext and its corresponding ciphertext. With this knowledge, the attacker attempts to infer the secret encryption key or devise a method for decrypting additional ciphertexts protected by the same key.
Once part of the keystream is exposed, the attacker can work towards recovering the structure of the LFSR.

A straightforward tactic would be to brute-force all potential initial seeds of the LFSR. However, due to the high computational cost, a more efficient approach is typically needed.
A fundamental technique for this recovery is the Berlekamp-Massey algorithm, which can find the LFSR's characteristic polynomial and its initial configuration. Acquiring these parameters allows the attacker to estimate the remainder of the keystream and thus decrypt future communications.

In this task the plaintext is provided by the file *known-plaintext.txt* and consists of a portion of decrypted text from the ciphertext *binary_sequence.bin*. 

In [22]:
# TEST KPA
# data loading
with open("known-plaintext.txt", "r", encoding="utf-8") as fp:
    known_pt = fp.read()
    
with open("ciphertext.bin", "rb") as fp:
    ct_bytes = fp.read()

pt_bits = Bits(known_pt.encode("utf-8"))
ct_bits = Bits(ct_bytes)

# known keystream
ks_known = pt_bits ^ Bits(ct_bits[:len(pt_bits)])

# Berlekamp–Massey on first 256 bits
poly_bits, L = berlekamp_massey(list(ks_known[:256]))
poly_set = {i for i, b in enumerate(poly_bits) if b}
print("Feedback poly:", poly_set, "  L =", L)

# initial state recovery
init_state = Bits(reversed(ks_known[:L]))

# full keystream generation
lfsr = LFSR(poly=poly_set, state=init_state)
present_output = Bits(lfsr.output)
keystream_full = present_output + lfsr.run_steps(len(ct_bits)-1)
recovered = (ct_bits ^ keystream_full).to_bytes()

print(recovered.decode("utf-8", errors="ignore"))

Feedback poly: {0, 1, 9, 48, 19}   L = 48
The Legacy of the Hidden Key

In a quiet corner of the university library, where dust motes danced in the slanted afternoon light, Jamie discovered an old, leather-bound journal tucked behind a row of forgotten books. Its cover bore no title, only an intricate emblem reminiscent of interlocking keys. Curiosity sparked, Jamie carefully opened the journal and found pages filled with cryptic symbols and strange, archaic notations--a mysterious cipher, perhaps long lost to time.

Intrigued by the enigma, Jamie began to study the symbols, recalling lessons from history about the Caesar cipher and other ancient methods of secret communication. The journal's faded ink hinted at a story of secrecy and rebellion from a bygone era. It described clandestine meetings, secret messages passed between revolutionaries, and whispered plans meant to outsmart an oppressive regime. Each line was a puzzle in itself--a coded window into a past where every secret cou

## Conclusion

The project successfully demonstrates the core principles behind a stream cipher based on an LFSR generator, clearly highlighting both its strengths and vulnerabilities.

The first key takeaway is how unpredictability and nonlinearity can be effectively achieved using a simple structure: by combining just a few LFSRs, it is possible to generate a complex output stream.

A strong solution able to successfully retrieve a suitable feedback polynomial has also been implemented. Starting from the output stream, the Berlekamp-Massey algorithm can be used for this goal.

Starting from these former concepts a complete Known-Plaintext Attack has been lastly demonstrated.
This result is an important conclusion to the project as it shows the vulnerabilty of using an LFSR generator for data encryption.

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e41f3657-3455-4db1-9261-9e3fe3459d95' target="_blank">

Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>